# Ingest Drivers.json file

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DataType
from pyspark.sql.functions import col, concat, current_timestamp, lit

#### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
"""This is a nested json file so we will define the inner schema (name_schema) and the outer schema separately seperately (drivers_schema)"""

Out[20]: 'This is a nested json file so we will define the inner schema (name_schema) and the outer schema separately seperately (drivers_schema)'

In [ ]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [ ]:
drivers_schema = StructType(fields=[
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), False),
    StructField("name", name_schema, False),
    StructField("url", StringType(), False),
])

In [ ]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json("/mnt/formula1lgdl/raw/drivers.json")

#### Step 2 - Rename columns and add new columns

In [ ]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("driverRef", "driver_ref") \
.withColumn("ingestion_date", current_timestamp()) \
.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))


#### Step 3 - Drop the unwanted columns

In [ ]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

#### Step 4 - Write to output to processed container in parquet format

In [ ]:
drivers_final_df.write.mode("overwrite").parquet("/mnt/formula1lgdl/processed/drivers")